# This is the part 1 implementation of the pdf

In [169]:
#this function calculates the distances between 2 vectors 
#the input feed is 2 np arrays
#np array[2,3,4,5]  and np array[1,4,8,2]
import numpy as np
from tqdm import tqdm
list1=np.array([2,1])
list2=np.array([-2,1])

####this is 1d euclidean distance

def d1_euc_dist(list1,list2):
    import numpy as np
    if (len(list1)==len(list2)):
        print("\n")
        #print("elements are of equal length, euc dist can be calculated")
    else:
        print("length of paths is not equal")
        return
    diff=(list1-list2)
    power=np.power(diff,2)
    total=np.sum(power)
    result=np.sqrt(total)
    return result

result=d1_euc_dist(list1,list2)
print("result:{}".format(result))



result:4.0


In [170]:
#this function calculates the distances between 2 vectors 
#the input feed is 2 np arrays
#np 2d array[2,3,4,5]  and np 2d array[1,4,8,2]
import numpy as np

list1=np.array([(2,1),(1,3)])
list2=np.ones((2,2),dtype=np.int16)

def euc_dist(list1,list2):
    """takes a 2d matrix and another 2d matrix and computes a result of distances"""
    if (list1.shape!=list2.shape):
        print("shapes do not match ip1 :{} and ip2: {}".format(list1.shape,list2.shape))
    diff=(list1-list2)
    power=np.power(diff,2)
    total=np.sum(power, axis=1)
    total=total.reshape((power.shape[0], 1))
    distances=np.power(total,(1/2))
    return distances

result=euc_dist(list1,list2)
print("result:{}".format(result))

result:[[1.]
 [2.]]


In [171]:
#calculateDistances is a function that takes 2d array of all the points and a query point 
#then returns distances with all the query points and sorted distances indexes
#no for loops inside this function

def calculateDistances(array2d, array1d):
    """array 2d is my all training points taken for calculation of distance
    array 1d is the query point with which the distance has to be calculated
    returns me distances sorted from top to bottom
    and their corresponding indices to look them in y_train"""
    #lets broadcast this query to all the points in the train 
    shape_arr1=array2d.shape
    #make ones of the same shape as train data is given
    broadcast_shape=np.ones(shape_arr1,dtype=np.int16)*array1d
    #now calculate the distance between the two arrays
    distances=euc_dist(broadcast_shape,array2d)
    #add an index to the distances
    rows=distances.shape[0] #counts froom 0
    index=np.arange(0,rows,1,dtype=np.int16)
    index=index.reshape((index.shape[0], 1))
    #stacking indexes with distances 
    index_dist=np.column_stack((index,distances))
    
    #now sorting the arrays based on 1st axis
    index_dist=index_dist[index_dist[:,1].argsort()]
    """returning the index value of the distance that can be used to look for the vector
    also returning the distance"""
    index=index_dist[:,0]
    #print(distances.shape)
    #distances=index_dist[:,1] #if we turn this on then we will only get the shortest value usually for k=1
    #shortest distances on the top
    return index, distances
 
#example run
array2d=np.array([(1.5,200,3), (1.5,3,30)], dtype = float)
array1d=np.array([1.5,2,3])

op1,op2=calculateDistances(array2d,array1d)    
print("result :{}, dist:{}".format(op1,op2))

result :[1. 0.], dist:[[198.        ]
 [ 27.01851217]]


In [172]:
def accuracy_score(arr1,arr2):
    """compares elements in both the arrays if they are same then its a match oterwise, No match
    return acc_score defined as total matches upon the elements present"""
    #since both of them are col vectors
    #we can take the row size as total number of elements
    total=arr1.shape[0]#row in arr1
    match=0
    for ele1,ele2 in zip(arr1,arr2):
        if(ele1==ele2):
            match+=1
    return match/total

pred_1=np.array([1,2,3,4])
pred_2=np.array([1,2,1,1])
score=accuracy_score(pred_1,pred_2)
print("score:{}".format(score))

score:0.5


# now load data to x_train, y_train, x_test,  y_test

In [173]:
#loading the train data
from numpy import genfromtxt
my_data = genfromtxt('trainingData.csv', delimiter=',')

print(my_data.shape)#you can check the data shape
#upon loading we found that the data has total 11 cols starting from 1 as the 1st col
#or you can say 10 cols starting from 0 as the 1st col
y_train=my_data[:,10]# this prints the labels
x_train=my_data[:,:-1] #every row except the last row until last col

#for test data
from numpy import genfromtxt
my_data = genfromtxt('testData.csv', delimiter=',')
y_test=my_data[:,10]
x_test=my_data[:,:-1]

#now lets start predicting a query point label
first=x_test[0] #this is the query point
#compute the KNN for first

indexes,result=calculateDistances(x_train,first)
print("indexes are :{}, distances are :{}".format(indexes,result))
#as we can see in the above example the 792th element in the train data is closest to the query point (oth in test data)
#and their distance value is lowest which is 0.13526

#lets try to verify this by actually picking up the 792th element and computing the distance of that point with query
no=792
val=d1_euc_dist(x_train[no],first)
print("the lowest distance computed between {} in train i.e {} and query{} is {}".format(no,x_train[no],first,val))

##in the above example we proved that this is actually the lowest query point
#since K=1 meaning we take only 1 nearest neighbour that means the nearest neighbour is the output
nn=int(indexes[0])#<<<this is my nearest neighbour index
#print(nn)
##look this neighbour in x_train
neighbour=x_train[nn]####<<<this is my neighbour returned
#print(neighbour)
#if we want to predict the class of this query we can look for the same index value in y_train and say this is the output

print("the label of the query point {} based on K= {} is label :{}".format(first,"1",y_train[nn]))

#can we now predict the values for all the elements in X_test
y_test_predicted=[]
print("="*50)
print("knn calculating..... for k=1")
for element in tqdm(x_test):
    index,distances=calculateDistances(x_train, element)
    #print("="*50)
    label=y_train[int(index[0])]
    #print(label)
    y_test_predicted.append(label)

y_test_predicted=np.asarray(y_test_predicted)
y_test_predicted.shape

acc=accuracy_score(y_test,y_test_predicted)*100
print("the accuracy value is :{}".format(acc))
print("="*50)

  7%|▋         | 69/1000 [00:00<00:01, 683.67it/s]

(4000, 11)
indexes are :[ 792. 1037. 1518. ... 2979.  534. 2068.], distances are :[[0.45730647]
 [0.40561773]
 [0.43496813]
 ...
 [0.32210837]
 [0.46737854]
 [0.86729724]]


the lowest distance computed between 792 in train i.e [0.51162893 0.47543597 0.38870508 0.49676188 0.5109842  0.2729233
 0.31984558 0.37737269 0.53117388 0.33281375] and query[0.54131721 0.52305685 0.42921551 0.39607615 0.52285322 0.27466181
 0.3682674  0.37434461 0.52591475 0.36184407] is 0.1352627341914311
the label of the query point [0.54131721 0.52305685 0.42921551 0.39607615 0.52285322 0.27466181
 0.3682674  0.37434461 0.52591475 0.36184407] based on K= 1 is label :0.0
knn calculating..... for k=1


100%|██████████| 1000/1000 [00:01<00:00, 690.63it/s]

the accuracy value is :89.5


100%|██████████| 1/1 [00:00<00:00,  7.31it/s]


89.5
